In [1]:
import math

import pandas

import psycopg2

from data_algebra.data_ops import *
import data_algebra.PostgreSQL

d = pandas.DataFrame({
    'x': [1, 2, math.inf, -math.inf, 2, 0]
})

d

,x
0,1.0
1,2.0
2,inf
3,-inf
4,2.0
5,0.0


In [2]:
ops = TableDescription("d", ["x"]) .\
    extend({
        "x_is_null": "x.is_null()",
        "x_is_bad": "x.is_bad()"
    })
ops

TableDescription(
 table_name='d',
 column_names=[
   'x']) .\
   extend({
    'x_is_null': 'x.is_null()',
    'x_is_bad': 'x.is_bad()'})

In [3]:
db_model_p = data_algebra.PostgreSQL.PostgreSQLModel()
sql_p = ops.to_sql(db_model_p, pretty=True)

print(sql_p)

SELECT "x",
       ("x" IS NULL) AS "x_is_null",
       ("x" IS NULL
        OR "x" >= '+infinity'
        OR "x" <= '-infinity'
        OR ("x" != 0
            AND "x" = -"x")) AS "x_is_bad"
FROM "d"


In [4]:
conn_p = psycopg2.connect(
    database="johnmount",
    user="johnmount",
    host="localhost",
    password=""
)
conn_p.autocommit=True

db_handle_p = db_model_p.db_handle(conn_p)

db_handle_p.insert_table(
    d,
    table_name='d',
    allow_overwrite=True)


res_p = db_handle_p.read_query(sql_p)

res_p

,x,x_is_null,x_is_bad
0,1.0,False,False
1,2.0,False,False
2,inf,False,True
3,-inf,False,True
4,2.0,False,False
5,0.0,False,False


In [5]:
db_handle_p.drop_table('d')
db_handle_p.close()
